In [3]:
import yfinance as yf

import os
os.chdir('C:\\Users\\btindol\\OneDrive - Stryker\\Linked In Learn\\Algorithmic Investing\Algorithmic Trading & Quantitative Analysis using Python')  
print(os.getcwd())  # Prints the current working directory

#####################################################################################################################################################
#imports 
#!pip install yfinance 
#!pip install pandas 
#!pip install numpy
#!pip install requests
!pip install datetime
#!pip install yahoofinancials
!pip install stocktrends
from yahoofinancials import YahooFinancials

import pandas as pd
import numpy as np
import requests
import datetime as dt
import os
import json 
from functools import reduce
import statsmodels.api as sm
from stocktrends import Renko


import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\btindol\OneDrive - Stryker\Linked In Learn\Algorithmic Investing\Algorithmic Trading & Quantitative Analysis using Python


You should consider upgrading via the 'c:\users\btindol\anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\btindol\anaconda3\python.exe -m pip install --upgrade pip' command.


# SINGLE STOCK ANALYZER NOW MAKE STRATEGY OFF OF IT

# CONSOLIDATION FOR EASE!!!!!!!!!!!!!!


In [4]:
# for each of the column names for the % change columns 

def shifted_column(df):
    pctchg = [col for col in df.columns if 'percent_change' in col] # grabe % change columns
    colnew = []                                                                    # initiate empty list
    for i in range(0,len(pctchg)):                                                 # loop over column names
        print(pctchg[i])
        var1 = df[pctchg[i]].shift(-1)                                            # grab colunmn by name and shift up one
        colnew.append(var1)                                                        # append to list
    colnew = pd.DataFrame(colnew)                                                  # now its wide dataframe
    colnew = colnew.transpose()                                                    # make it narrow
    cols = colnew.columns.tolist()                                                 # take column names of narrow dataframe                                                
    newcols = []                                                                   # make list to add the text shift to the column name
    for j in range(0,len(cols)): 
        colz = cols[j] + "_" + "shift"                                             # add shift to the name of shifted column

        newcols.append(colz) 
    colnew.columns = newcols  
    return colnew
    #########################################################################################
    #########################################################################################
    
def clean_up_zeros_and_column_names(df):
    df =df.fillna(0)     
    df.columns = df.columns.str.replace("-", "_") 
    return df

In [5]:
def get_daily_returns(df):
    daily_return = df.pct_change()
    cols = daily_return.columns.tolist()
    newcols = []
    for j in range(0,len(cols)):
        if cols[j] == "formatted_date":
            colz = cols[j]
        else:
            colz = cols[j] + "_" + "percent_change"

        newcols.append(colz)
    daily_return.columns = newcols               
    return daily_return

In [6]:
def get_current_position(df,initial_position):
    
    # Calculate returns from initial position entry 
    shiftval = [col for col in df.columns if 'shift' in col]                   # find the shift column to bring % change up to today to mutliply to get positions next day amount.. 

    initial_position = initial_position                                             # pass from function above

    new = [(df[shiftval[0]][0] * initial_position) + initial_position]        # get first value in final for iterate the 0 of this 
    for i in range(1, len(df.index)):                                          # starting at second row of the dataframe column 
        new.append((new[i-1]*df[shiftval].values[i][0]) +new[i-1])
    
    df['Position'] = new
    df['Position'] = df['Position'].shift(1)
    df['Position'][0] = initial_position
    df.fillna(0)
    return df#['Position']

In [7]:
# Calculating RSI without using loop
def rsi(df, n):
    "function to calculate RSI"
    delta = df["Adj Close"].diff().dropna()
    u = delta * 0
    d = u.copy()
    u[delta > 0] = delta[delta > 0]
    d[delta < 0] = -delta[delta < 0]
    u[u.index[n-1]] = np.mean( u[:n]) # first value is average of gains
    u = u.drop(u.index[:(n-1)])
    d[d.index[n-1]] = np.mean( d[:n]) # first value is average of losses
    d = d.drop(d.index[:(n-1)])
    rs = u.ewm(com=n,min_periods=n).mean()/d.ewm(com=n,min_periods=n).mean()
    return 100 - 100 / (1+rs)

In [8]:
def ATR(DF,n):
    "function to calculate True Range and Average True Range"
    df = DF.copy()
    df['H-L']=abs(df['High']-df['Low'])
    df['H-PC']=abs(df['High']-df['Adj Close'].shift(1))
    df['L-PC']=abs(df['Low']-df['Adj Close'].shift(1))
    df['TR']=df[['H-L','H-PC','L-PC']].max(axis=1,skipna=False)
    df['ATR'] = df['TR'].rolling(n).mean()
    #df['ATR'] = df['TR'].ewm(span=n,adjust=False,min_periods=n).mean()
    df2 = df.drop(['H-L','H-PC','L-PC'],axis=1)
    df2.dropna(inplace=True)
    return df2

In [9]:
def ADX(DF,n):
    "function to calculate ADX"
    df2 = DF.copy()
    df2['TR'] = ATR(df2,n)['TR'] #the period parameter of ATR function does not matter because period does not influence TR calculation
    df2['DMplus']=np.where((df2['High']-df2['High'].shift(1))>(df2['Low'].shift(1)-df2['Low']),df2['High']-df2['High'].shift(1),0)
    df2['DMplus']=np.where(df2['DMplus']<0,0,df2['DMplus'])
    df2['DMminus']=np.where((df2['Low'].shift(1)-df2['Low'])>(df2['High']-df2['High'].shift(1)),df2['Low'].shift(1)-df2['Low'],0)
    df2['DMminus']=np.where(df2['DMminus']<0,0,df2['DMminus'])
    TRn = []
    DMplusN = []
    DMminusN = []
    TR = df2['TR'].tolist()
    DMplus = df2['DMplus'].tolist()
    DMminus = df2['DMminus'].tolist()
    for i in range(len(df2)):
        if i < n:
            TRn.append(np.NaN)
            DMplusN.append(np.NaN)
            DMminusN.append(np.NaN)
        elif i == n:
            TRn.append(df2['TR'].rolling(n).sum().tolist()[n])
            DMplusN.append(df2['DMplus'].rolling(n).sum().tolist()[n])
            DMminusN.append(df2['DMminus'].rolling(n).sum().tolist()[n])
        elif i > n:
            TRn.append(TRn[i-1] - (TRn[i-1]/n) + TR[i])
            DMplusN.append(DMplusN[i-1] - (DMplusN[i-1]/n) + DMplus[i])
            DMminusN.append(DMminusN[i-1] - (DMminusN[i-1]/n) + DMminus[i])
    df2['TRn'] = np.array(TRn)
    df2['DMplusN'] = np.array(DMplusN)
    df2['DMminusN'] = np.array(DMminusN)
    df2['DIplusN']=100*(df2['DMplusN']/df2['TRn'])
    df2['DIminusN']=100*(df2['DMminusN']/df2['TRn'])
    df2['DIdiff']=abs(df2['DIplusN']-df2['DIminusN'])
    df2['DIsum']=df2['DIplusN']+df2['DIminusN']
    df2['DX']=100*(df2['DIdiff']/df2['DIsum'])
    ADX = []
    DX = df2['DX'].tolist()
    for j in range(len(df2)):
        if j < 2*n-1:
            ADX.append(np.NaN)
        elif j == 2*n-1:
            ADX.append(df2['DX'][j-n+1:j+1].mean())
        elif j > 2*n-1:
            ADX.append(((n-1)*ADX[j-1] + DX[j])/n)
    df2['ADX']=np.array(ADX)
    return df2['ADX']

In [10]:
def slope(ser,n):
    "function to calculate the slope of regression line for n consecutive points on a plot"
    ser = (ser - ser.min())/(ser.max() - ser.min())
    x = np.array(range(len(ser)))
    x = (x - x.min())/(x.max() - x.min())
    slopes = [i*0 for i in range(n-1)]
    for i in range(n,len(ser)+1):
        y_scaled = ser[i-n:i]
        x_scaled = x[:n]
        x_scaled = sm.add_constant(x_scaled)
        model = sm.OLS(y_scaled,x_scaled)
        results = model.fit()
        slopes.append(results.params[-1])
    slope_angle = (np.rad2deg(np.arctan(np.array(slopes))))
    return np.array(slope_angle)



# Come back for Renko Later!!


In [11]:
def OBV(DF):
    """function to calculate On Balance Volume"""
    df = DF.copy()
    df['daily_ret'] = df['Adj Close'].pct_change()
    df['direction'] = np.where(df['daily_ret']>=0,1,-1)
    df['direction'][0] = 0
    df['vol_adj'] = df['Volume'] * df['direction']
    df['obv'] = df['vol_adj'].cumsum()
    return df['obv']


In [127]:
def single_stock_analyzer(stocks, begin_date,end_date,initial_position,slow_ma,fast_ma):

    # looping over tickers and creating a dataframe with closing prices
    # get just adjusted close column for that ticker
    for ticker in stocks:
        AdjclosePrice = yf.download(ticker,beg_date,end_date)[['Adj Close','Open','High','Low','Close','Volume']]     
    
    # Make it a dataframe
    AdjclosePrice = pd.DataFrame(AdjclosePrice)
    
    # pass dataframe into function that gets the adjusted close percent change column
    AdjclosePrice_PctChg = get_daily_returns(AdjclosePrice)
    
    # merge percent change column to normal price of the stock (Adj Close)
    final = AdjclosePrice.merge(AdjclosePrice_PctChg, on='Date', how='left')
    
    # Shift the percent change column up for multiplying by the initial position input column later to simulate earnings.. 
    colnew =shifted_column(AdjclosePrice_PctChg)

    # join the new column onto existing Adj Close and Adj Close percent change columns 
    final_f= final.merge(colnew, on='Date', how='left')

    # fill na's with 0's and change weird characters such as - to _ in column names 
    final_f = clean_up_zeros_and_column_names(final_f)

    # See how the money you put in performed.. 
    var = get_current_position(final_f,initial_position)
    final_f['Position'] = var
    
    # Get the moving averages for the crossover strategt slow and fast moving averages..  
    final_f['Fast_moving_average'] = final_f['Adj Close'].rolling(window=fast_ma).mean() # 5 day price moving average
    final_f['Slow_moving_average'] = final_f['Adj Close'].rolling(window=slow_ma).mean() # 5 day price moving average

    # Bolligner band of the fast moving average 30 day.. 
    final_f["BB_up"] = final_f["Fast_moving_average"] + 2*final_f['Adj Close'].rolling(fast_ma).std(ddof=0) #ddof=0 is required since we want to take the standard deviation of the population and not sample
    final_f["BB_dn"] = final_f["Fast_moving_average"] - 2*final_f['Adj Close'].rolling(fast_ma).std(ddof=0) #ddof=0 is required since we want to take the standard deviation of the population and not sample
    final_f["BB_width"] = final_f["BB_up"] - final_f["BB_dn"]
    
    # rsi standard 14 periods
    final_f["RSI_fast_14_periods"] = rsi(final_f,14) # standard periods is 14
    final_f["RSI_slow_30_periods"] = rsi(final_f,30) # longer than average periods 30 
    
    # Average true range 14 period is standard 
    final_f2 = ATR(final_f,14) # 14 is standard for longer use 20 
    
    # Average directional movement here we will use a period matching the fast moving average
    final_f2['ADX'] = ADX(final_f2,fast_ma)
    
    
    #MACD, short for moving average convergence/divergence    
    """function to calculate MACD
       typical values a = 12; b =26, c =9"""
    a = 12;b=26;c=9
    final_f2["MA_Fast"]=final_f2["Adj Close"].ewm(span=a,min_periods=a).mean()
    final_f2["MA_Slow"]=final_f2["Adj Close"].ewm(span=b,min_periods=b).mean()
    final_f2["MACD"]=final_f2["MA_Fast"]-final_f2["MA_Slow"]
    final_f2["Signal"]=final_f2["MACD"].ewm(span=c,min_periods=c).mean()
    
    
    # Slope for last 5 days change if needed
    final_f2["close_slope"] = slope(final_f2["Adj Close"],5)
    
    # OBV
    OBV2= OBV(final_f2)
    
    # Merge OBV 
    final_f3 = final_f2.merge(OBV2, on='Date', how='left')
    
    return final_f3

In [253]:
stocks = ['ETH-USD']
end_date = (dt.date.today()).strftime('%Y-%m-%d')
beg_date = (dt.date.today()-dt.timedelta(1825)).strftime('%Y-%m-%d') # 1825 is 5 years ish 
final_f = single_stock_analyzer(stocks,beg_date,end_date,10.239700,30,90)
final_f

[*********************100%***********************]  1 of 1 completed
Adj Close_percent_change
Open_percent_change
High_percent_change
Low_percent_change
Close_percent_change
Volume_percent_change


,Adj Close,Open,High,Low,Close,Volume,Adj Close_percent_change,Open_percent_change,High_percent_change,Low_percent_change,...,RSI_slow_30_periods,TR,ATR,ADX,MA_Fast,MA_Slow,MACD,Signal,close_slope,obv
Date,,,,,,,,,,,,,,,,,,,,,
2016-08-15,11.218400,11.201700,11.310800,11.000100,11.218400,7500840,0.002108,-0.032944,-0.032819,-0.008920,...,46.438092,0.310699,0.821903,NaN,NaN,NaN,NaN,NaN,0.000000,0
2016-08-16,11.139700,11.228700,11.237300,11.002800,11.139700,7051510,-0.007015,0.002410,-0.006498,0.000245,...,46.192026,0.234500,0.628923,NaN,NaN,NaN,NaN,NaN,0.000000,-7051510
2016-08-17,10.752800,11.133200,11.154800,10.748600,10.752800,8833760,-0.034732,-0.008505,-0.007342,-0.023103,...,44.981230,0.406200,0.541457,NaN,NaN,NaN,NaN,NaN,0.000000,-15885270
2016-08-18,10.756100,10.747500,11.009100,10.744800,10.756100,6935800,0.000307,-0.034644,-0.013062,-0.000354,...,44.993937,0.264300,0.479279,NaN,NaN,NaN,NaN,NaN,0.000000,-8949470
2016-08-19,10.748700,10.746300,10.801100,10.727900,10.748700,5390850,-0.000688,-0.000112,-0.018893,-0.001573,...,44.969870,0.073200,0.454993,NaN,NaN,NaN,NaN,NaN,-3.149577,-14340320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-12,3785.848633,4174.635742,4362.350586,3785.848633,3785.848633,69023382175,-0.091840,0.057332,0.044072,0.000518,...,68.170133,576.501953,308.350638,NaN,3579.782941,3139.604429,440.178513,374.486826,14.589988,1645837355450
2021-05-13,3715.148438,3828.918457,4032.563477,3549.407227,3715.148438,78398214539,-0.018675,-0.082814,-0.075598,-0.062454,...,66.750814,483.156250,333.871390,NaN,3600.608402,3182.237318,418.371084,383.263678,-87.681484,1567439140911
2021-05-14,4079.057373,3720.122314,4171.017090,3703.399902,4079.057373,48174271215,0.097953,-0.028414,0.034334,0.043385,...,70.065693,467.617188,362.423689,NaN,3674.215936,3248.668433,425.547503,391.720443,-83.145903,1615613412126


# LEFT OFF HERE !!! TRYING TO IMPLEMENT STRATEGY!!

# Get crossover strategy here then get multiple crossovers.. 

In [285]:
def sma_strategy(df,short_window,long_window):
    short_window= 30
    long_window = 90
    final_f = df.copy()

    CrossOver_Strat = final_f[["Adj Close"]]

    # Short and long moving average.. 
    CrossOver_Strat["short_mavg"] = CrossOver_Strat["Adj Close"].rolling(window= short_window,min_periods=1, center=False).mean()
    CrossOver_Strat["long_mavg"] = CrossOver_Strat["Adj Close"].rolling(window= long_window,min_periods=1, center=False).mean()

    CrossOver_Strat["signal"]=0
    CrossOver_Strat["signal"][short_window:] =np.where(CrossOver_Strat['short_mavg'][short_window:] > CrossOver_Strat['long_mavg'][short_window:],1,0)

    CrossOver_Strat["positions"] = CrossOver_Strat["signal"].diff()
    CrossOver_Strat["positions"] = CrossOver_Strat["positions"].fillna(0)

    CrossOver_Strat['buy_only'] = np.where(CrossOver_Strat['positions'] == 1,1000,0) # buy every day if 1
    CrossOver_Strat['buy_sell'] = np.where(CrossOver_Strat['positions'] == 1,1000,np.where(CrossOver_Strat['positions']==-1,-1000,0)) # buy right when it crosses over.. 

    CrossOver_Strat['percent_change_shift'] = CrossOver_Strat["Adj Close"].pct_change().shift(-1) # get percent change for adj close and shift it up for multiplication.
    CrossOver_Strat["percent_change_shift"] = CrossOver_Strat["percent_change_shift"].fillna(0)
    return CrossOver_Strat

var = sma_strategy(final_f,30,90)
var

,Adj Close,short_mavg,long_mavg,signal,positions,buy_only,buy_sell,percent_change_shift
Date,,,,,,,,
2016-08-15,11.218400,11.218400,11.218400,0,0.0,0,0,-0.007015
2016-08-16,11.139700,11.179050,11.179050,0,0.0,0,0,-0.034732
2016-08-17,10.752800,11.036967,11.036967,0,0.0,0,0,0.000307
2016-08-18,10.756100,10.966750,10.966750,0,0.0,0,0,-0.000688
2016-08-19,10.748700,10.923140,10.923140,0,0.0,0,0,0.046396
...,...,...,...,...,...,...,...,...
2021-05-12,3785.848633,2890.551660,2166.595882,1,0.0,0,0,-0.018675
2021-05-13,3715.148438,2937.750350,2187.391614,1,0.0,0,0,0.097953
2021-05-14,4079.057373,2992.548763,2212.557697,1,0.0,0,0,-0.108097


In [255]:
def buy_at_beginning_and_hold(CrossOver_Strat,initial_position):
    initial_position = initial_position
    new = [(CrossOver_Strat.percent_change_shift.values[0] * initial_position) + initial_position] # make the first value of new column hold it in variable where we take initial position and multiply by the shifted percent change column to get initial starting value
    for i in range(1, len(CrossOver_Strat.index)): # starting at second row of the dataframe column 
        new.append((new[i-1]*CrossOver_Strat.percent_change_shift.values[i]) +new[i-1]) # take new and multiply the first calculated value by the next percent change plus the origional value to get new position value

    # Assign the list new to the dataframe column
    CrossOver_Strat['Buy_at_Begining_and_Hold_Position'] = new
    CrossOver_Strat['Buy_at_Begining_and_Hold_Position'] = CrossOver_Strat['Buy_at_Begining_and_Hold_Position'].round(2).shift(1) # shift it down to make since since shifted percent change up one value
    CrossOver_Strat.Buy_at_Begining_and_Hold_Position.iloc[0] = initial_position  # inject the initial starting amount into the first value of the column.. 
    return CrossOver_Strat

var2 = buy_at_beginning_and_hold(var,11.218400)
var2

,Adj Close,short_mavg,long_mavg,signal,positions,buy_only,buy_sell,percent_change_shift,Buy_at_Begining_and_Hold_Position
Date,,,,,,,,,
2016-08-15,11.218400,11.218400,11.218400,0,0.0,0,0,-0.007015,11.2184
2016-08-16,11.139700,11.179050,11.179050,0,0.0,0,0,-0.034732,11.1400
2016-08-17,10.752800,11.036967,11.036967,0,0.0,0,0,0.000307,10.7500
2016-08-18,10.756100,10.966750,10.966750,0,0.0,0,0,-0.000688,10.7600
2016-08-19,10.748700,10.923140,10.923140,0,0.0,0,0,0.046396,10.7500
...,...,...,...,...,...,...,...,...,...
2021-05-12,3785.848633,2890.551660,2166.595882,1,0.0,0,0,-0.018675,3785.8500
2021-05-13,3715.148438,2937.750350,2187.391614,1,0.0,0,0,0.097953,3715.1500
2021-05-14,4079.057373,2992.548763,2212.557697,1,0.0,0,0,-0.108097,4079.0600


In [256]:
import random

In [257]:
def get_random_list_of_indexes(df):
    additional = df[df['positions'] == 1.0].sum().positions # get the sum of positions and minus the first for the entry amount.. 
    additional= (additional-1).astype(int) # minus 1 becuase when comparing when you initialize at first cross over you dont add additional to first 

    #additional_amount = 1000
     # Make random list of certain amount of variables as many as there are buys
    randomlist = []
    for i in range(0,additional):
        n = random.randint(1,1732)
        randomlist.append(n)
    randomlist.sort()
    return randomlist

randomlist = get_random_list_of_indexes(var2)
randomlist

[89, 365, 399, 984, 1192, 1474, 1575, 1710]

In [258]:
def additional_amount_column(df,randomlist):
    for i in range(0,len(randomlist)):
        newindex = randomlist[i]
        df.loc[df.index[newindex], 'additional_contribution'] = additional_amount

    df['additional_contribution'][0] = initial_amount
    df['additional_contribution']  =var2['additional_contribution'].fillna(0)  
    df.additional_contribution.count()
    return df

varf = additional_amount_column(var2,randomlist)
varf

,Adj Close,short_mavg,long_mavg,signal,positions,buy_only,buy_sell,percent_change_shift,Buy_at_Begining_and_Hold_Position,additional_contribution
Date,,,,,,,,,,
2016-08-15,11.218400,11.218400,11.218400,0,0.0,0,0,-0.007015,11.2184,100.0
2016-08-16,11.139700,11.179050,11.179050,0,0.0,0,0,-0.034732,11.1400,0.0
2016-08-17,10.752800,11.036967,11.036967,0,0.0,0,0,0.000307,10.7500,0.0
2016-08-18,10.756100,10.966750,10.966750,0,0.0,0,0,-0.000688,10.7600,0.0
2016-08-19,10.748700,10.923140,10.923140,0,0.0,0,0,0.046396,10.7500,0.0
...,...,...,...,...,...,...,...,...,...,...
2021-05-12,3785.848633,2890.551660,2166.595882,1,0.0,0,0,-0.018675,3785.8500,0.0
2021-05-13,3715.148438,2937.750350,2187.391614,1,0.0,0,0,0.097953,3715.1500,0.0
2021-05-14,4079.057373,2992.548763,2212.557697,1,0.0,0,0,-0.108097,4079.0600,0.0


In [259]:
varf[varf['additional_contribution'] > 0].count().additional_contribution # This works.. 

9

In [261]:

# Mkke colunmn that gets the value of current position for the additional contribution strategy.. 
def buy_beginning_randomly_contribute_same_number_of_times_as_strategy(df):
    initial_position = df['additional_contribution'][0]
    new = [(df.percent_change_shift.values[0] * initial_position) + initial_position]
    for i in range(1, len(df.index)): # starting at second row of the dataframe column 
         new.append((new[i-1]*df.percent_change_shift.values[i]) + new[i-1] +df.additional_contribution.values[i])# take new and multiply the first calculated value by the next percent change plus the origional value to get new position value
    df['buy_at_begin_and_random_contribute'] = new
    return df
dff =buy_beginning_randomly_contribute_same_number_of_times_as_strategy(varf)
dff

,Adj Close,short_mavg,long_mavg,signal,positions,buy_only,buy_sell,percent_change_shift,Buy_at_Begining_and_Hold_Position,additional_contribution,buy_at_begin_and_random_contribute
Date,,,,,,,,,,,
2016-08-15,11.218400,11.218400,11.218400,0,0.0,0,0,-0.007015,11.2184,100.0,99.298473
2016-08-16,11.139700,11.179050,11.179050,0,0.0,0,0,-0.034732,11.1400,0.0,95.849675
2016-08-17,10.752800,11.036967,11.036967,0,0.0,0,0,0.000307,10.7500,0.0,95.879089
2016-08-18,10.756100,10.966750,10.966750,0,0.0,0,0,-0.000688,10.7600,0.0,95.813130
2016-08-19,10.748700,10.923140,10.923140,0,0.0,0,0,0.046396,10.7500,0.0,100.258506
...,...,...,...,...,...,...,...,...,...,...,...
2021-05-12,3785.848633,2890.551660,2166.595882,1,0.0,0,0,-0.018675,3785.8500,0.0,489456.896597
2021-05-13,3715.148438,2937.750350,2187.391614,1,0.0,0,0,0.097953,3715.1500,0.0,537400.536329
2021-05-14,4079.057373,2992.548763,2212.557697,1,0.0,0,0,-0.108097,4079.0600,0.0,479308.961118


In [266]:

def counter_column(df):
    newft = []
    counter = 0
    for i in range(0, len(df.index)):
        if df.signal.values[i] == 0 and counter == 0:
            newft.append(0)
        else:
            counter +=1
            newft.append(counter)
    return  newft  

dff['counter'] =counter_column(dff)
dff

,Adj Close,short_mavg,long_mavg,signal,positions,buy_only,buy_sell,percent_change_shift,Buy_at_Begining_and_Hold_Position,additional_contribution,buy_at_begin_and_random_contribute,counter
Date,,,,,,,,,,,,
2016-08-15,11.218400,11.218400,11.218400,0,0.0,0,0,-0.007015,11.2184,100.0,99.298473,0
2016-08-16,11.139700,11.179050,11.179050,0,0.0,0,0,-0.034732,11.1400,0.0,95.849675,0
2016-08-17,10.752800,11.036967,11.036967,0,0.0,0,0,0.000307,10.7500,0.0,95.879089,0
2016-08-18,10.756100,10.966750,10.966750,0,0.0,0,0,-0.000688,10.7600,0.0,95.813130,0
2016-08-19,10.748700,10.923140,10.923140,0,0.0,0,0,0.046396,10.7500,0.0,100.258506,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-12,3785.848633,2890.551660,2166.595882,1,0.0,0,0,-0.018675,3785.8500,0.0,489456.896597,1698
2021-05-13,3715.148438,2937.750350,2187.391614,1,0.0,0,0,0.097953,3715.1500,0.0,537400.536329,1699
2021-05-14,4079.057373,2992.548763,2212.557697,1,0.0,0,0,-0.108097,4079.0600,0.0,479308.961118,1700


In [294]:
# MAKE COLUMN WHERE YOU ARE TAKING THE INITAL POSITION AT FIRST CROSSOVER THEN YOU ARE ADDING 1000 DOLLARS TO EACH BUY SIGNAL AFTER.. 
# MAKE SURE THAT THE BUY SIGNAL IS THE SAME NUMBER OF BUYS AS THE RANDOMLY SPACED SCENARIO SO ITS EVEN.. 

initial_position = 100
varzf = dff[dff.positions == 1].first_valid_index()    # find first strategy entry position
dff["Position_Start"] = np.where(dff.index == varzf,initial_position,0) # find first time that position is activated.. 
dff["buy_only_final"] = np.where(dff.index == varzf,0,dff["buy_only"])  # we will only buy not sell
dff["inital_position_at_first_crossover_buy_only"] = np.where(dff.index == varzf,initial_position,dff["buy_only_final"])
dff.head(50)
#dff.to_csv("comz.csv")

filtered_df = dff.loc[varzf::,]  # bottom half after first signal.. 

# Top half with no calculation.. 
length = len(dff.loc[:varzf,])-1
filtered_df2 = dff.iloc[:length,] 

initial_position = 100
new = [(filtered_df.percent_change_shift.values[0] * initial_position) + initial_position]
for i in range(1, len(filtered_df.index)): # starting at second row of the dataframe column 
     new.append((new[i-1]*filtered_df.percent_change_shift.values[i]) + new[i-1] +filtered_df.inital_position_at_first_crossover_buy_only.values[i])# take new and multiply the first calculated value by the next percent change plus the origional value to get new position value
filtered_df['crossover_buy_only_final'] = new
filtered_df['crossover_buy_only_final'].replace(np.nan, 0)
filtered_df.head(50)


final_df = pd.concat([filtered_df2, filtered_df])
#final_df.to_csv("comzz.csv")
final_df

,Adj Close,short_mavg,long_mavg,signal,positions,buy_only,buy_sell,percent_change_shift,Buy_at_Begining_and_Hold_Position,additional_contribution,buy_at_begin_and_random_contribute,counter,buy_only_final,Position_Start,inital_position_at_first_crossover_buy_only,crossover_buy_only_final
Date,,,,,,,,,,,,,,,,
2016-08-15,11.218400,11.218400,11.218400,0,0.0,0,0,-0.007015,11.2184,100.0,99.298473,0,0,0,0,NaN
2016-08-16,11.139700,11.179050,11.179050,0,0.0,0,0,-0.034732,11.1400,0.0,95.849675,0,0,0,0,NaN
2016-08-17,10.752800,11.036967,11.036967,0,0.0,0,0,0.000307,10.7500,0.0,95.879089,0,0,0,0,NaN
2016-08-18,10.756100,10.966750,10.966750,0,0.0,0,0,-0.000688,10.7600,0.0,95.813130,0,0,0,0,NaN
2016-08-19,10.748700,10.923140,10.923140,0,0.0,0,0,0.046396,10.7500,0.0,100.258506,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-12,3785.848633,2890.551660,2166.595882,1,0.0,0,0,-0.018675,3785.8500,0.0,489456.896597,1698,0,0,0,474506.374858
2021-05-13,3715.148438,2937.750350,2187.391614,1,0.0,0,0,0.097953,3715.1500,0.0,537400.536329,1699,0,0,0,520985.570155
2021-05-14,4079.057373,2992.548763,2212.557697,1,0.0,0,0,-0.108097,4079.0600,0.0,479308.961118,1700,0,0,0,464668.409329


# NOW MAKE COLUMN WHERE YOU BUY 1000 WHEN 1 AND sell all of your stock or a certain percentage at -1..
# Then rebuy all you have again when hits 1.. 



for i in range(0,len(df)):
    when turns first singal buy.
    if turns 1 then add 1000 $ 
    if turns -1 then exit all of trade and sell all stock
    if turns 1 again then buy with all of money you have again. 
    
    

In [316]:
returnzvar = (filtered_df['Adj Close'][0] * filtered_df['percent_change_shift'][0]) +filtered_df['Adj Close'][0]
returnzvar

11.937000274658203

In [323]:
(filtered_df.iloc[0][0]  *filtered_df.iloc[0][7]) + filtered_df.iloc[0][0]


11.937000274658203

In [355]:
cash = []
returnz = []
initial_position = 100
varzf = filtered_df[filtered_df.positions == 1].first_valid_index() 
filtered_df = filtered_df.loc[varzf::,]  # bottom half after first signal.. 


In [414]:
balance =100# how much you have on you balance
balancehold =[]
cash = [] # this is how much cash you have invested in now
returnz = [] # list of returns
initial_position = 100
varzf = filtered_df[filtered_df.positions == 1].first_valid_index() 
filtered_df = filtered_df.loc[varzf::,]  # bottom half after first signal.. 

for i in range(0,len(filtered_df)):
    if i == 0: # if it is the first value of dataframe where we filter to the first crossover as the first row
        cash.append(initial_position) # append to cashposition list the initial entry position as first value 
        returnzvar = (filtered_df.iloc[i][0]  * filtered_df.iloc[i][7]) + (filtered_df.iloc[i][0] + initial_position) # multiply the price or adjusted value by the % change and add initial position plus price.. (NOt sure if i should do this??? no multiply for first posotiion??)
        returnz.append(returnzvar)
        balance = returnz[i]
        balancehold.append(balance)
        
    elif filtered_df.positions[i] == 1: # if its not the first time the cross over happnes (aka) when we put initial position down
        
        cash.append(5000)
        returnz.append(5000)
        balance += 5000
        balancehold.append(balance)

    elif filtered_df.positions[i] == -1:
        cashval = -100
        cash.append(cashcashval)
        returnval = returnz[i-1]-100 # hold for later with cash 
        returnz.append(returnval)
        balance -= 5000
        balancehold.append(balance)

    elif filtered_df.positions[i] == 0:
        cash.append(cash[i-1])
    # Basically then we take the previous value times tjhe return plus previous value
        returnzvar = (returnz[i-1] * filtered_df.iloc[i][7]) + returnz[i-1] # take last returnz value then multiply it 
        returnz.append(returnzvar)
        balance = returnz[i-1] 
        balancehold.append(balance)

    else:
        continue
        
    

# THIS IS CLOSE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [424]:
# THIS IS GETTING CLOSE TO HOW WE COULD DO IT!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# NEED TO SELL ALL OF THE CASH POSITION WHEN THERE IS A -1 AND THEN BUY ALL OF THE MONEY IN BALANCE WHERE HAVE A BUY POSITION AGAIN .. 



balance =100# how much you have on you balance
balancehold =[]
cash = [] # this is how much cash you have invested in now
returnz = [] # list of returns
initial_position = 100
varzf = filtered_df[filtered_df.positions == 1].first_valid_index() 
filtered_df = filtered_df.loc[varzf::,]  # bottom half after first signal.. 

for i in range(0,len(filtered_df)):
    if i == 0: # if it is the first value of dataframe where we filter to the first crossover as the first row
        cash.append(initial_position) # append to cashposition list the initial entry position as first value 
        returnzvar = (filtered_df.iloc[i][0]  * filtered_df.iloc[i][7]) + (filtered_df.iloc[i][0] + initial_position) # multiply the price or adjusted value by the % change and add initial position plus price.. (NOt sure if i should do this??? no multiply for first posotiion??)
        returnz.append(returnzvar)
    
        
    elif filtered_df.positions[i] == 1: # if its not the first time the cross over happnes (aka) when we put initial position down
        
        cash.append(cash[i-1] + 5000)
        returnz.append(returnz[i-1] +5000)


    elif filtered_df.positions[i] == -1:
        cashval = 100
        cash.append(cash[i-1] - cashval)
        returnval = returnz[i-1]-100 # hold for later with cash 
        returnz.append(returnval)


    elif filtered_df.positions[i] == 0:
        cash.append(cash[i-1])
    # Basically then we take the previous value times tjhe return plus previous value
        returnzvar = (returnz[i-1] * filtered_df.iloc[i][7]) + returnz[i-1] # take last returnz value then multiply it 
        returnz.append(returnzvar)


    else:
        continue
        

In [426]:
cash

[100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,
 5000,

In [422]:
# var2 = [balancehold,returnz]
# var2 = pd.DataFrame(var2)
# #df1.T # or df1.transpose()
# var2 =var2.T
# var2.head(50)

In [420]:
cash = []
returnz = []
initial_position = 100
varzf = filtered_df[filtered_df.positions == 1].first_valid_index() 
filtered_df = filtered_df.loc[varzf::,]  # bottom half after first signal.. 

for i in range(0,len(filtered_df)):
    if i == 0: # if it is the first value of dataframe where we filter to the first crossover as the first row
        cash.append(initial_position) # append to cashposition list the initial entry position as first value 
        returnzvar = (filtered_df.iloc[i][0]  * filtered_df.iloc[i][7]) + (filtered_df.iloc[i][0] + initial_position) # multiply the price or adjusted value by the % change and add initial position plus price.. (NOt sure if i should do this??? no multiply for first posotiion??)
        returnz.append(returnzvar)
        
    elif filtered_df.positions[i] == 0: # if its not the first time the cross over happnes (aka) when we put initial position down
        
    # Basically then we take the previous value times tjhe return plus previous value
        returnzvar = (returnz[i-1] * filtered_df.iloc[i][7]) + returnz[i-1] # take last returnz value then multiply it 
        returnz.append(returnzvar)
        
        # If there si no buy aka if positions == 0 then we just say the current value of the cash wallet is the previous value which is initial posit
        cashposition = cash[i-1] 
        cash.append(cashposition)
    else:
        cash.append(5000)
        returnz.append(returnz[i-1] + 5000)

    


In [378]:
returnzvar

41168.21157394519

In [421]:
returnz

[111.9370002746582,
 117.73968373382306,
 119.07875834085095,
 116.57126655537557,
 123.67739609379677,
 135.31745253502453,
 129.11155144027657,
 124.22690174576717,
 125.01178461936075,
 120.76011363862382,
 122.85125066319068,
 120.24623651442498,
 122.82687230809398,
 124.48197124102315,
 122.59337309621776,
 124.01310375042725,
 123.52454398619861,
 123.7777354717076,
 126.14738134175363,
 124.50260251952939,
 122.23985117570152,
 120.53880360429592,
 118.78337423635952,
 114.58233797556296,
 113.0313344400386,
 110.23501369837915,
 110.51820862532226,
 111.90417981860364,
 112.18081065433582,
 111.9482504473331,
 112.37210830432603,
 112.0739071852141,
 112.57465612699109,
 118.0463180109894,
 112.73969741213762,
 113.4561223590015,
 114.17254730586538,
 113.19824479202858,
 112.86535415001859,
 112.23707046061361,
 107.03265830651607,
 108.1391816494448,
 107.83629444886893,
 103.94846549935313,
 97.94229521264974,
 104.86463019716514,
 103.13357918432543,
 100.95710003111303,
 

In [214]:
# How many sells do we have
CrossOver_Strat3 = CrossOver_Strat[CrossOver_Strat['positions'] == -1.0]
CrossOver_Strat3['positions'].sum() # There were 8 sells of 1000$

-8.0

In [153]:
# Compare trading methods..
#Add 8*1000 to first inital position and call it buy and hold..
# buy 1000 at 1 where 30 is greater than 90 (long_ma) and sell 1000 when -1 
# then buy all you have at 1 and sell all you have cumulatively at -1..
# then start off  with regular initial amount at get in the same amount of money at arbitrary random time interval no strategy..
# Also enter at the beginning and just add when its time.

# If minus 1000 add 2000 at next position?? 

In [222]:
initial_position = 100                                                       # initialize the first amount to add to trade
var = CrossOver_Strat[CrossOver_Strat.positions == 1].first_valid_index()    # find first strategy entry position
var #'2016-10-27'
CrossOver_Strat["Position_Start"] = np.where(CrossOver_Strat.index == var,initial_position,0) # FIND the first time it crosses over to buy

# We dont want to start the position adding additional 1000 because we want to put our intial amount in there
CrossOver_Strat["buy_final"] = np.where(CrossOver_Strat.index == var,0,CrossOver_Strat["buy"])                          # make column that is the same as buy where first time we add were adding 1000 and inital
CrossOver_Strat["inital_position_at_first_crossover"] = np.where(CrossOver_Strat.index == var,initial_position,CrossOver_Strat["buy_final"]) # first position dont add position only but from there add the buys. 


# We dont want to start the position adding additional 1000 because we want to put our intial amount in there
CrossOver_Strat["buy_only_final"] = np.where(CrossOver_Strat.index == var,0,CrossOver_Strat["buy_only"])                          # make column that is the same as buy where first time we add were adding 1000 and inital
CrossOver_Strat["inital_position_at_first_crossover_buy_only"] = np.where(CrossOver_Strat.index == var,initial_position,CrossOver_Strat["buy_only_final"]) # first position dont add position only but from there add the buys. 



In [223]:
CrossOver_Strat.head(50)

,Adj Close,short_mavg,long_mavg,signal,positions,buy_only,buy,percent_change_shift,CurrentPosition,initial_position_at_beginning,Position_Start,buy_final,inital_position_at_first_crossover,buy_only_final,inital_position_at_first_crossover_buy_only,newpos
Date,,,,,,,,,,,,,,,,
2016-08-15,11.2184,11.218400,11.218400,0,0.0,0,0,-0.007015,11.2184,100,0,0,0,0,0,-0.000000
2016-08-16,11.1397,11.179050,11.179050,0,0.0,0,0,-0.034732,11.1400,0,0,0,0,0,0,-0.000000
2016-08-17,10.7528,11.036967,11.036967,0,0.0,0,0,0.000307,10.7500,0,0,0,0,0,0,0.000000
2016-08-18,10.7561,10.966750,10.966750,0,0.0,0,0,-0.000688,10.7600,0,0,0,0,0,0,-0.000000
2016-08-19,10.7487,10.923140,10.923140,0,0.0,0,0,0.046396,10.7500,0,0,0,0,0,0,0.000000
2016-08-20,11.2474,10.977183,10.977183,0,0.0,0,0,-0.006064,11.2500,0,0,0,0,0,0,-0.000000
2016-08-21,11.1792,11.006043,11.006043,0,0.0,0,0,-0.005215,11.1800,0,0,0,0,0,0,-0.000000
2016-08-22,11.1209,11.020400,11.020400,0,0.0,0,0,-0.008120,11.1200,0,0,0,0,0,0,-0.000000
2016-08-23,11.0306,11.021533,11.021533,0,0.0,0,0,0.000490,11.0300,0,0,0,0,0,0,0.000000


In [348]:
newft = []
counter = 0
for i in range(0, len(CrossOver_Strat.index)):
    if CrossOver_Strat.signal.values[i] == 0 and counter == 0:
        newft.append(0)
    else:
        counter +=1
        newft.append(counter)
        
CrossOver_Strat['newft'] = newft
CrossOver_Strat

,Adj Close,short_mavg,long_mavg,signal,positions,buy_only,buy,percent_change_shift,CurrentPosition,initial_position_at_beginning,Position_Start,buy_final,inital_position_at_first_crossover,buy_only_final,inital_position_at_first_crossover_buy_only,newpos,new,newft
Date,,,,,,,,,,,,,,,,,,
2016-08-15,11.218400,11.218400,11.218400,0,0.0,0,0,-0.007015,11.2184,100,0,0,0,0,0,-0.0,0.000000,0
2016-08-16,11.139700,11.179050,11.179050,0,0.0,0,0,-0.034732,11.1400,0,0,0,0,0,0,-0.0,0.000000,0
2016-08-17,10.752800,11.036967,11.036967,0,0.0,0,0,0.000307,10.7500,0,0,0,0,0,0,0.0,0.000000,0
2016-08-18,10.756100,10.966750,10.966750,0,0.0,0,0,-0.000688,10.7600,0,0,0,0,0,0,-0.0,0.000000,0
2016-08-19,10.748700,10.923140,10.923140,0,0.0,0,0,0.046396,10.7500,0,0,0,0,0,0,0.0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-13,3715.148438,2937.750350,2187.391614,1,0.0,0,0,0.097953,3715.1500,0,0,0,0,0,0,0.0,520985.570155,1699
2021-05-14,4079.057373,2992.548763,2212.557697,1,0.0,0,0,-0.108097,4079.0600,0,0,0,0,0,0,-0.0,464668.409329,1700
2021-05-15,3638.122070,3029.848958,2232.924786,1,0.0,0,0,-0.013913,3638.1200,0,0,0,0,0,0,-0.0,458203.634282,1701


In [359]:
#newval2

In [331]:
initial_position = 100                                                       # initialize the first amount to add to trade
var = CrossOver_Strat[CrossOver_Strat.positions == 1].first_valid_index()    # find first strategy entry position
var #'2016-09-14'
length = len(CrossOver_Strat)
filtered_df = CrossOver_Strat.loc[var::,]                                          # start at the beginning of the position..
filtered_df

,Adj Close,short_mavg,long_mavg,signal,positions,buy_only,buy,percent_change_shift,CurrentPosition,initial_position_at_beginning,Position_Start,buy_final,inital_position_at_first_crossover,buy_only_final,inital_position_at_first_crossover_buy_only,newpos,new,newft
Date,,,,,,,,,,,,,,,,,,
2016-09-14,11.923200,11.422283,11.415706,1,1.0,1000,1000,0.001157,11.92,0,100,0,100,0,100,0.115746,100.115746,1
2016-09-15,11.937000,11.448860,11.431997,1,0.0,0,0,0.051839,11.94,0,0,0,0,0,0,0.000000,105.305629,2
2016-09-16,12.555800,11.508960,11.466052,1,0.0,0,0,0.011373,12.56,0,0,0,0,0,0,0.000000,106.503289,3
2016-09-17,12.698600,11.573710,11.502303,1,0.0,0,0,-0.021057,12.70,0,0,0,0,0,0,-0.000000,104.260604,4
2016-09-18,12.431200,11.629793,11.528843,1,0.0,0,0,0.060960,12.43,0,0,0,0,0,0,0.000000,110.616282,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-13,3715.148438,2937.750350,2187.391614,1,0.0,0,0,0.097953,3715.15,0,0,0,0,0,0,0.000000,520985.570155,1015
2021-05-14,4079.057373,2992.548763,2212.557697,1,0.0,0,0,-0.108097,4079.06,0,0,0,0,0,0,-0.000000,464668.409329,1016
2021-05-15,3638.122070,3029.848958,2232.924786,1,0.0,0,0,-0.013913,3638.12,0,0,0,0,0,0,-0.000000,458203.634282,1017


In [318]:
filtered_df['inital_position_at_first_crossover_buy_only'].sum()

8100

In [313]:
initial_position = 100                                                       # initialize the first amount to add to trade
var = CrossOver_Strat[CrossOver_Strat.positions == 1].first_valid_index()    # find first strategy entry position
var#'2016-09-14'
length = len(CrossOver_Strat.loc[:var,])-1
filtered_df2 = CrossOver_Strat.iloc[:length,]                                          # Get everything before the entry posoition. 
filtered_df2.head()

,Adj Close,short_mavg,long_mavg,signal,positions,buy_only,buy,percent_change_shift,CurrentPosition,initial_position_at_beginning,Position_Start,buy_final,inital_position_at_first_crossover,buy_only_final,inital_position_at_first_crossover_buy_only,newpos,new
Date,,,,,,,,,,,,,,,,,
2016-08-15,11.2184,11.218400,11.218400,0,0.0,0,0,-0.007015,11.2184,100,0,0,0,0,0,-0.0,0.0
2016-08-16,11.1397,11.179050,11.179050,0,0.0,0,0,-0.034732,11.1400,0,0,0,0,0,0,-0.0,0.0
2016-08-17,10.7528,11.036967,11.036967,0,0.0,0,0,0.000307,10.7500,0,0,0,0,0,0,0.0,0.0
2016-08-18,10.7561,10.966750,10.966750,0,0.0,0,0,-0.000688,10.7600,0,0,0,0,0,0,-0.0,0.0
2016-08-19,10.7487,10.923140,10.923140,0,0.0,0,0,0.046396,10.7500,0,0,0,0,0,0,0.0,0.0


In [312]:
# Add the extra amount then get the return! # Buy only 
initial_position = 100
initial_position =initial_position
new = [(filtered_df.percent_change_shift.values[0] * initial_position) + initial_position]
for i in range(1, len(filtered_df.index)): # starting at second row of the dataframe column 
     new.append((new[i-1]*filtered_df.percent_change_shift.values[i]) + new[i-1] +filtered_df.inital_position_at_first_crossover_buy_only.values[i])# take new and multiply the first calculated value by the next percent change plus the origional value to get new position value
filtered_df['new'] = new
filtered_df.head(50)

filtered_df.to_csv("new.csv")
#filtered_df['new2'] = np.where(filtered_df['inital_position_at_first_crossover_buy_only'] != 0, (filtered_df['buy_only_final'] + filtered_df['new']) *
#filtered_df.head(50)

In [316]:
final_df = pd.concat([filtered_df2, filtered_df])
final_df
final_df.to_csv("new2.csv")

In [317]:
final_df.head(50)

,Adj Close,short_mavg,long_mavg,signal,positions,buy_only,buy,percent_change_shift,CurrentPosition,initial_position_at_beginning,Position_Start,buy_final,inital_position_at_first_crossover,buy_only_final,inital_position_at_first_crossover_buy_only,newpos,new,new2
Date,,,,,,,,,,,,,,,,,,
2016-08-15,11.2184,11.218400,11.218400,0,0.0,0,0,-0.007015,11.2184,100,0,0,0,0,0,-0.000000,0.000000,NaN
2016-08-16,11.1397,11.179050,11.179050,0,0.0,0,0,-0.034732,11.1400,0,0,0,0,0,0,-0.000000,0.000000,NaN
2016-08-17,10.7528,11.036967,11.036967,0,0.0,0,0,0.000307,10.7500,0,0,0,0,0,0,0.000000,0.000000,NaN
2016-08-18,10.7561,10.966750,10.966750,0,0.0,0,0,-0.000688,10.7600,0,0,0,0,0,0,-0.000000,0.000000,NaN
2016-08-19,10.7487,10.923140,10.923140,0,0.0,0,0,0.046396,10.7500,0,0,0,0,0,0,0.000000,0.000000,NaN
2016-08-20,11.2474,10.977183,10.977183,0,0.0,0,0,-0.006064,11.2500,0,0,0,0,0,0,-0.000000,0.000000,NaN
2016-08-21,11.1792,11.006043,11.006043,0,0.0,0,0,-0.005215,11.1800,0,0,0,0,0,0,-0.000000,0.000000,NaN
2016-08-22,11.1209,11.020400,11.020400,0,0.0,0,0,-0.008120,11.1200,0,0,0,0,0,0,-0.000000,0.000000,NaN
2016-08-23,11.0306,11.021533,11.021533,0,0.0,0,0,0.000490,11.0300,0,0,0,0,0,0,0.000000,0.000000,NaN


In [ ]:
# This is using the total amount (bought only no selling) and buying at the beginning and holding.  
# IF YOU HAD THAT MUCH MONEY.. 
startval =filtered_df['inital_position_at_first_crossover_buy_only'].sum() # make column where you use this as the start

In [322]:
# Normal position without using the strategy.. 
startval =filtered_df['inital_position_at_first_crossover_buy_only'].sum() # make column where you use this as the start
initial_position = startval
new = [(final_df.percent_change_shift.values[0] * initial_position) + initial_position] # make the first value of new column hold it in variable where we take initial position and multiply by the shifted percent change column to get initial starting value
for i in range(1, len(final_df.index)): # starting at second row of the dataframe column 
    new.append((new[i-1]*final_df.percent_change_shift.values[i]) +new[i-1]) # take new and multiply the first calculated value by the next percent change plus the origional value to get new position value

# Assign the list new to the dataframe column
final_df['start_full_no_strat'] = new
final_df['start_full_no_strat'] = final_df['start_full_no_strat'].round(2).shift(1) # shift it down to make since since shifted percent change up one value
final_df.start_full_no_strat.iloc[0] = startval  # inject the initial starting amount into the first value of the column.. 
final_df.to_csv("new3.csv")

# Try to incorporate the selling intot the equation.. not just buying
Compare adding random amounts where you buy in randomly spaced intervals to buying initial value at first.
# Compare trading methods..
#Add 8*1000 to first inital position and call it buy and hold..
# buy 1000 at 1 where 30 is greater than 90 (long_ma) and sell 1000 when -1 
# then buy all you have at 1 and sell all you have cumulatively at -1..
# then start off  with regular initial amount at get in the same amount of money at arbitrary random time interval no strategy..
# Also enter at the beginning and just add when its time.

# If minus 1000 add 2000 at next position?? 

In [265]:
CrossOver_Strat.inital_position_at_first_crossover_buy_only.values[0]

0

In [51]:
var = CrossOver_Strat[CrossOver_Strat.positions == 1].first_valid_index()
var

Timestamp('2016-09-14 00:00:00')

In [147]:
# var = add_strategy(CrossOver_Strat,100)
# var.head(50)

In [ ]:
# start off with 

In [ ]:
# Now add 100 at the begining vs adding 100 at the positions..
# Also add one where you buy at a specified interval once a month..
# First count up the number of times/ amount that you buy.. 

In [47]:
# Make new column if index is the first time ther is a cross over then add the value in..

#MAKE SURE TO SHIFT THE FINAL OUTCOME COLUMN BY THE RIGHT AMOUNT 

initial_position = 100000
var = CrossOver_Strat[CrossOver_Strat.positions == 1].first_valid_index()
var #'2016-10-27'
CrossOver_Strat["Position_Start"] = np.where(CrossOver_Strat.index == var,initial_position,0) # FIND the first time it crosses over to buy
CrossOver_Strat["Position_plus_buy"] = CrossOver_Strat["buy"] + CrossOver_Strat["Position_Start"]
CrossOver_Strat["Current_Position_Base"] = CrossOver_Strat["Position_plus_buy"].cumsum()
CrossOver_Strat["Current_Position_Pct_Change"] =np.where(CrossOver_Strat['signal'] == 1,CrossOver_Strat["Current_Position_Base"] * CrossOver_Strat["percent_change_shift"],0)
CrossOver_Strat['Current_Position_Final2'] =CrossOver_Strat["Current_Position_Base"] + CrossOver_Strat["Current_Position_Pct_Change"]
CrossOver_Strat['Current_Position_Final3'] =np.where(CrossOver_Strat["signal"] == 0,0,CrossOver_Strat['Current_Position_Final2'])


# NEED TO FIND A WAY TO FIND O8T 
#CrossOver_Strat['cumsum_buy'] = CrossOver_Strat["buy"].cumsum()
#CrossOver_Strat["Current_Position_Pct_Change_cumbuy"] =np.where(CrossOver_Strat['signal'] == 1,CrossOver_Strat["Current_Position_Base"] * CrossOver_Strat["percent_change_shift"],0)
CrossOver_Strat.head(50)

,Adj Close,short_mavg,long_mavg,signal,positions,buy,percent_change_shift,CurrentPosition,Current_Position_Final,Position_Start,Position_plus_buy,Current_Position_Base,Current_Position_Pct_Change,Current_Position_Final2,Current_Position_Final3
Date,,,,,,,,,,,,,,,
2016-08-15,11.2184,11.218400,11.218400,0,0.0,0,-0.007015,99.30,100.00,0,0,0,0.000000,0.000000,0.000000
2016-08-16,11.1397,11.179050,11.179050,0,0.0,0,-0.034732,95.85,99.30,0,0,0,0.000000,0.000000,0.000000
2016-08-17,10.7528,11.036967,11.036967,0,0.0,0,0.000307,95.88,95.85,0,0,0,0.000000,0.000000,0.000000
2016-08-18,10.7561,10.966750,10.966750,0,0.0,0,-0.000688,95.81,95.88,0,0,0,0.000000,0.000000,0.000000
2016-08-19,10.7487,10.923140,10.923140,0,0.0,0,0.046396,100.26,95.81,0,0,0,0.000000,0.000000,0.000000
2016-08-20,11.2474,10.977183,10.977183,0,0.0,0,-0.006064,99.65,100.26,0,0,0,0.000000,0.000000,0.000000
2016-08-21,11.1792,11.006043,11.006043,0,0.0,0,-0.005215,99.13,99.65,0,0,0,0.000000,0.000000,0.000000
2016-08-22,11.1209,11.020400,11.020400,0,0.0,0,-0.008120,98.33,99.13,0,0,0,0.000000,0.000000,0.000000
2016-08-23,11.0306,11.021533,11.021533,0,0.0,0,0.000490,98.37,98.33,0,0,0,0.000000,0.000000,0.000000


# MAKE SURE THE PERCENT CHANGE LOG ACCUMULATION OF RETURN MAKES SENSE TOO!! BEFORE MAKING SURE YOU HAVE WORKING STRATEGY THEN IMPLEMENT TRADING COSTS.. 